<a href="https://colab.research.google.com/github/Nwanna-Joseph/nlp_week_1_solution/blob/week3/Copy_of_n_gram_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<h1 style="font-family:verdana;font-size:300%;text-align:center;background-color:#f2f2f2;color:#0d0d0d">AMMI NLP - Review sessions</h1>

<h1 style="font-family:verdana;font-size:180%;text-align:Center;color:#993333"> Lab 3: n-gram models </h1>

**Big thanks to Amr Khalifa who improved this lab and made it to a Jupyter Notebook!**

In [ ]:
import io, sys, math, re
from collections import defaultdict
import numpy as np

In [ ]:
# data_loader
def load_data(filename):
    '''
    parameters:
    filename (string): datafile
    
    Returns:
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    '''
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    vocab = defaultdict(lambda:0)
    for line in fin:
        sentence = line.split()
        data.append(sentence)
        for word in sentence:
            vocab[word] += 1
    return data, vocab

In [ ]:
print("load training set..")
print("\n")
train_data, vocab = load_data("train1.txt")
print(train_data[0])
print("\n")
print("how :",vocab['how'])
print("load validation set")
valid_data, _ = load_data("valid1.txt")


load training set..


['<s>', 'my', 'fathers', "don't", 'speak', 'dutch.', '</s>']


how : 107
load validation set


In [ ]:
vocab

In [ ]:
len(train_data)

5561

In [ ]:
def remove_rare_words(data, vocab, mincount = 1):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    mincount(int): the minimum count 
    
    Returns: 
    data_with_unk(list of lists): data after replacing rare words with <unk> token
    '''
    # replace words in data that are not in the vocab 
    # or have a count that is below mincount
    data_with_unk = []
    print()
    ## FILL CODE
    for each_sentence in data:
      sentence = each_sentence
      sentence_copy = []
      for each_word in sentence:
        word = each_word
        word_frequency = vocab.get(word)

        if(word_frequency == None):
          sentence_copy.append('<unk>')
          continue
        
        if ( word_frequency < mincount):
          sentence_copy.append('<unk>')
          continue

        sentence_copy.append(word)

      
      data_with_unk.append(sentence_copy)

    return data_with_unk

In [ ]:
print("remove rare words")
train_data = remove_rare_words(train_data, vocab, mincount = 1)
valid_data = remove_rare_words(valid_data, vocab, mincount = 1)
#train_data
print(train_data[0])


remove rare words


['<s>', 'my', 'fathers', "don't", 'speak', 'dutch.', '</s>']


In [ ]:
train_data

In [ ]:
def build_ngram(data, n):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    n (int): size of the n-gram
    
    Returns:
    prob (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    '''
    total_number_words = 0
    counts = defaultdict(lambda: defaultdict(lambda: 0.0))

    for sentence in data:
        sentence = tuple(sentence)
        ## FILL CODE
        # dict can be indexed by tuples
        # store in the same dict all the ngrams
        # by using the context as a key and the word as a value
        counts[sentence[0:1]] [sentence[1]] +=1

        for index in range(( len(sentence) - n  ) ):
          counts[sentence[index:index+n]] [sentence[index+n]] +=1
          counts[sentence[index+1:index+n]] [sentence[index+n]] +=1
        print(sentence, counts.keys())
        break

    prob = defaultdict(lambda: defaultdict(lambda: 0.0))
    # Build the probabilities from the counts
    # Be careful with how you normalize!

    for context in counts.keys():
    ## FILL CODE
      context_words_frequencies = counts[context].values()
      context_total_appearance = sum(context_words_frequencies)
      

      for entry in counts[context]:
        # print(context, entry , counts[context][entry],"/",context_total_appearance)
        prob[context][entry] = counts[context][entry] / context_total_appearance


    return prob

In [ ]:
# RUN TO BUILD NGRAM MODEL

n = 2
print("build ngram model with n = ", n)
model = build_ngram(train_data, n)

build ngram model with n =  2
('<s>', 'my', 'fathers', "don't", 'speak', 'dutch.', '</s>') dict_keys([('<s>',), ('<s>', 'my'), ('my',), ('my', 'fathers'), ('fathers',), ('fathers', "don't"), ("don't",), ("don't", 'speak'), ('speak',), ('speak', 'dutch.'), ('dutch.',)])


In [ ]:
model


defaultdict(<function __main__.build_ngram.<locals>.<lambda>>,
            {('<s>',): defaultdict(<function __main__.build_ngram.<locals>.<lambda>.<locals>.<lambda>>,
                         {'my': 1.0}),
             ('<s>',
              'my'): defaultdict(<function __main__.build_ngram.<locals>.<lambda>.<locals>.<lambda>>, {'fathers': 1.0}),
             ("don't",): defaultdict(<function __main__.build_ngram.<locals>.<lambda>.<locals>.<lambda>>,
                         {'speak': 1.0}),
             ("don't",
              'speak'): defaultdict(<function __main__.build_ngram.<locals>.<lambda>.<locals>.<lambda>>, {'dutch.': 1.0}),
             ('dutch.',): defaultdict(<function __main__.build_ngram.<locals>.<lambda>.<locals>.<lambda>>,
                         {'</s>': 1.0}),
             ('fathers',): defaultdict(<function __main__.build_ngram.<locals>.<lambda>.<locals>.<lambda>>,
                         {"don't": 1.0}),
             ('fathers',
              "don't"): defaultdict

Here, implement a recursive function over shorter and shorter context to compute a "stupid backoff model". An interpolation model can also be implemented this way.

In [ ]:
def get_prob(model, context, w):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    } 
    context (list of strings): a sentence
    w(string): the word we need to find it's probability given the context
    
    Retunrs:
    prob(float): probability of this word given the context 
    '''

    # code a recursive function over 
    # smaller and smaller context
    # to compute the backoff model
    
    ## FILL CODE

    length = len(context)
    backoff = 1

    for i in range(len(context)):
      prob = model.get([context[i:]],{}).get([w])

      if prob == None:
        backoff * 0.4
      else:
        return prob * backoff
        

    return backoff * model[w][w]

In [ ]:
def perplexity(model, data, n):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    } 
    data (list of lists): each list is a sentence of the text
    n(int): size of the n-gram
    
    Retunrs:
    perp(float): the perplexity of the model 
    '''
    ## FILL CODE
    prob = 1
    for each_sentence in data:
      for each_word in each_sentence:
        prob += get_prob(model, each_sentence, each_word, n)

    return prob

In [ ]:
valid_data

In [ ]:
# COMPUTE PERPLEXITY ON VALIDATION SET

print("The perplexity is", perplexity(model, valid_data, n=n))

TypeError: ignored

In [ ]:
def get_proba_distrib(model, context):
    ## need to get the the words after the context and their probability of appearance
    ## after this context 
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    context (list of strings): the sentence we need to find the words after it and 
    thier probabilites
    
    Retunrs:
    words_and_probs(dic): {word: probability of word given context}
    
    '''
    # code a recursive function over context
    # to find the longest available ngram
    
    ## FILL CODE
    for start in range( len(context) - n ):
      for new_n in range( len(context) - start - 1,0, -1):
        context_key = context[start:start+new_n]
        map = model[context_key]
        if map == None:
          continue
        
        return model[context_key]
    
    return None

In [ ]:
def generate(model):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    
    Retunrs:
    sentence (list of strings): a sentence sampled according to the language model. 
    

    '''
    # generate a sentence. A sentence starts with a <s> and ends with a </s>
    # Possiblly a use function is:
    # np.random.choice(x, 1, p = y)

    # where x is a list of things to sample from
    # and y is a list of probability (of the same length as x)
    sentence = ["<s>"]
    while sentence[-1] != "</s>" and len(sentence)<100:
        key = sentence[-1]
        nexts = model[key]
        choice = np.random.choice(nexts.keys(), 1, p = nexts.values())
        sentence.append(choice)
        
    return sentence

In [ ]:
print(model['<s>'])

defaultdict(<function build_ngram.<locals>.<lambda>.<locals>.<lambda> at 0x7f98664569e0>, {})


In [ ]:
# GENERATE A SENTENCE FROM THE MODEL

print("Generated sentence: ",generate(model))

ValueError: ignored

Once you are done implementing the model, evaluation and generation code, you can try changing the value of `n`, and play with a larger training set (`train2.txt` and `valid2.txt`). You can also try to implement an interpolation model.